In [1]:
'''
import packages we need
'''
import json
import requests
import os

In [2]:
import csv

In [3]:
import osmnx as ox

In [4]:
'''
open an empty csv for further use
'''
def csv_save(row):
    with open("street_2.0.csv", 'a') as f:
        write = csv.writer(f)
        write.writerow(row)

In [5]:
'''
find the township that the community address belongs to 
'''
def get_street(x,y):
    url_0 = 'https://restapi.amap.com/v3/geocode/regeo?'
    location1 = str(x)#"113.275128"
    location2 = str(y)#"23.117633"
    key = 'a1cf852d9781d1e427a06ab57400ce13'
    radius="10"
    extensions = 'all'
    url = url_0 + 'key=' + key + '&location=' + location1 + ',' + location2  + '&radius=' + radius + '&city=' + '&extensions=' + extensions
    html = requests.get(url)
    content = json.loads(html.text)
    street = content["regeocode"]['addressComponent']['township']
    return street

In [15]:
'''
using osmnx to get intersection_density_km and street_length_avg of the address
'''
def osm(x):
    try:
        G = ox.graph_from_place(x, which_result=None)
    except (ValueError):  
        G = ox.graph_from_address(x, dist=500)
    G_proj = ox.project_graph(G)
    nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
    graph_area_m = nodes_proj.unary_union.convex_hull.area
    try:
        basic_stats=ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist="euclidean")
    except (ZeroDivisionError):
        G = ox.graph_from_address(x, dist=500)
        G_proj = ox.project_graph(G)
        nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
        graph_area_m = nodes_proj.unary_union.convex_hull.area
        basic_stats=ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist="euclidean")
        
    j=[]
    j.append(basic_stats['intersection_density_km'])
    j.append(basic_stats['street_length_avg'])
    return j

        

In [33]:
'''
addresses are from this file
'''
file='address_gz.txt'
f=open(file,'r')
ff=f.readlines()

In [34]:
'''
write information into correspond lines with elegance
'''
for line in ff:
    i =[]
    line=line.rstrip("\n")
    #print(line)
    i.append(str(line))
    line = line.split(',')
    #print(line)
    x = float(line[0])
    y = float(line[1])
    street = get_street(x,y)
    i.append(str(street))
    i.append(osm(street))
    print(i)
    csv_save(i)

['113.377729,22.962841', '东环街道', [34.55350248171042, 151.0652597402597]]
['113.168921,23.373785', '秀全街道', [67.60406911970202, 136.25749999999996]]
['113.382736,23.162493', '长兴街道', [26.45506325515186, 139.07963249516445]]
['113.485525,23.577278', '鳌头镇', [2.178182158374352, 390.61106521739066]]
['113.411500,23.157736', '新塘街道', [22.902008083435057, 177.12659677419379]]
['113.278753,23.118938', '珠光街道', [182.1003530684274, 79.21584647302905]]
['113.304949,23.236708', '永平街道', [41.842903207089684, 112.75554814108695]]
['113.466782,23.008770', '化龙镇', [11.934894725405652, 238.04456203007533]]


In [36]:
'''
codes down below are used to debug
'''
x

'云埔街道'

In [40]:
G = ox.graph_from_address(x, dist=500)

In [41]:
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
graph_area_m = nodes_proj.unary_union.convex_hull.area
basic_stats=ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist="euclidean")

In [42]:
basic_stats

{'n': 44,
 'm': 81,
 'k_avg': 3.6818181818181817,
 'intersection_count': 43,
 'streets_per_node_avg': 3.3863636363636362,
 'streets_per_node_counts': {0: 0, 1: 1, 2: 0, 3: 24, 4: 19},
 'streets_per_node_proportion': {0: 0.0,
  1: 0.022727272727272728,
  2: 0.0,
  3: 0.5454545454545454,
  4: 0.4318181818181818},
 'edge_length_total': 8578.875999999998,
 'edge_length_avg': 105.91204938271603,
 'street_length_total': 6803.921,
 'street_length_avg': 106.311265625,
 'street_segments_count': 64,
 'node_density_km': 90.01886882878954,
 'intersection_density_km': 87.97298544631705,
 'edge_density_km': 17551.379848692057,
 'street_density_km': 13920.028909555604,
 'circuity_avg': 1.0393791165680217,
 'self_loop_proportion': 0.0,
 'clean_intersection_count': 20,
 'clean_intersection_density_km': 40.91766764944979}